Выпускная квалификационная работа по курсу «Data Science». Тема: Прогнозирование конечных свойств новых материалов (композиционных материалов).

In [ ]:
# импорт неободимых библиотек
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import matplotlib
%matplotlib inline 
matplotlib.style.use('seaborn-talk')

from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization

In [ ]:
# загрузка первого датасета (базальтопластик)
df_bp = pd.read_excel('data/X_bp.xlsx')

In [ ]:
# просмотр данных,содержащихся в датасете
df_bp.sample(5)

In [ ]:
# размерность датасета
df_bp.shape

In [ ]:
# удалю столбец "Unnamed: 0", т.к. он не несет в себе данных, необходимых для дальнейшего исследования
df_bp.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
# загрузка второго датасета (углепластик)
df_up = pd.read_excel('data/X_nup.xlsx')

In [ ]:
df_up.sample(5)

In [ ]:
# размерность датасета
df_up.shape

In [ ]:
df_up.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
# объединю по индексу два датасета, тип объединения INNER
df = df_bp.merge(df_up, left_index=True, right_index=True, how='inner')
df.head()

Разведочный анализ:

In [ ]:
# сделаю визуальную оценку данных и проведу разведочный анализ
df

In [ ]:
df.info()

In [ ]:
# из таблицы с информацией о датасете видно, что в наборе данных отстуствует строковый тип данных, нет пропусков, отсутствуют значения NaN.
# дополнительные операции по очистке данных не требуются

In [ ]:
df.profile_report()

In [ ]:
# отчет показывает множество характеристик, в том числе среднее и медиалнное значение,  а так же гистограммы распределений.В датасете отсутствуют дубликаты.  
# по гистограммам видно, что данные стремятся к нормальному распределению, кроме признака "Угол нашивки", где данные имеют всего два значения. 

In [ ]:
df.columns

In [ ]:
df.corr()

In [ ]:
corr=df.corr()
plt.figure(figsize=(10,7))
sns.heatmap(corr,annot=True, fmt='.1g',  cmap= 'viridis')
plt.title('Correlation Matrix')
plt.show()

In [ ]:
# дополню анализ диаграммами ящика с усами, чтобы оценить наличие выбросов
fig=plt.figure(figsize=(35,35))
for i, col in enumerate(df.columns):
    ax=fig.add_subplot(4, 4, i+1)
    sns.boxplot(data=df, y=df[col], fliersize=15, linewidth=5)
plt.title(col, size = 20)
plt.show()

In [ ]:
# наличие выбросов прослеживается во всех признаках,кроме "Угол нашивки,град"

In [ ]:
# строю попарные графики рассеяния точек
sns.pairplot(df)

In [ ]:
# на графиках так же явно видны выбросы, точки, которые удалены от наибольшего скопления значений.
# линейная зависимость между признаками не прослеживается

Предобработка данных:

In [ ]:
# так как все признаки , кроме "Угол нашивки" имеют распределение , близкое к нормальному, то для определения выбросов воспользуюсь методом трех сигм:
count_3S = 0
for column in df:
  d = df.loc[:, [column]]
  zscore = (df[column] - df[column].mean()) / df[column].std()
  d['3S'] = zscore.abs() > 3
  count_3S += d['3S'].sum()
print('Количество выбросов методом трех сигм:', count_3S)

In [ ]:
#удаление выбросов:
df_clean = df[(np.abs(stats.zscore(df)) <= 3).all(axis = 1)]

In [ ]:
df_clean.shape

In [ ]:
#повторно проверю выбросы 
count_3S = 0
for column in df_clean:
  d = df_clean.loc[:, [column]]
  zscore = (df_clean[column] - df_clean[column].mean()) / df_clean[column].std()
  d['3S'] = zscore.abs() > 3
  count_3S += d['3S'].sum()
print('Количество выбросов методом трех сигм:', count_3S)

In [ ]:
#удаление выбросов:
df_clean = df_clean[(np.abs(stats.zscore(df_clean)) <= 3).all(axis = 1)]

In [ ]:
#повторно проверю выбросы 
count_3S = 0
for column in df_clean:
  d = df_clean.loc[:, [column]]
  zscore = (df_clean[column] - df_clean[column].mean()) / df_clean[column].std()
  d['3S'] = zscore.abs() > 3
  count_3S += d['3S'].sum()
print('Количество выбросов методом трех сигм:', count_3S)

In [ ]:
df_clean.iplot()

In [ ]:
#график показывает, что данные имеют широкий диапазон значению, поэтому необходимо сделать нормализацию данных.
# выполню нормализацию при помощи MinMaxScaler

In [ ]:
Mmsclr = MinMaxScaler()
df_norm = pd.DataFrame(Mmsclr.fit_transform(df_clean), columns = df_clean.columns, index = df_clean.index)

In [ ]:
df_norm.iplot()

In [ ]:
# после применения MinMaxScaler все значения теперь расположены в диапозоне от 0 до 1

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
df_norm.plot(kind = 'kde', ax = ax)

In [ ]:
# на графике видно, что все данные нормально распределены, кроме признака "поверхностная плотность, г/м2" - он смещен влево и "угол нашивки, град" это категориальный признак, т.к. имеет всего 2 уникадьных значения

In [ ]:
# описательная статистика для очищенных и нормальзованных данных

In [ ]:
df_norm.profile_report()

In [ ]:
sns.pairplot(df_norm, height=4, diag_kind='kde', corner = True);

In [ ]:
#на обработанных данных линейная зависимость между переменными также не прослеживается

Обучение моделей

In [ ]:
df_norm.columns

1. Модель для прогноза модуля упругости при растяжении

In [ ]:
X = df_norm.drop('Модуль упругости при растяжении, ГПа', axis = 1)
y = df_norm['Модуль упругости при растяжении, ГПа']

In [ ]:
# делю данные на обучающую и тестовую выборку, 70% и 30% 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)
print(f'Количество записей в обучающей выборке: {X_train.shape[0]}')
print(f'Количество записей в тестовой выборке: {X_test.shape[0]}')

In [ ]:
# функция для применения поиска гиперпараметров модели с помощью поиска по сетке с перекрестной проверкой (GridSearchCV)
# функция для вывода результатов работы модели

In [ ]:
def GridSCV(model, params, X_train, y_train):
  GSCV = GridSearchCV(model, params, n_jobs=-1, cv=10)  
  GSCV.fit(X_train, y_train)
  GSCV.best_params_
  print(f'Лучший параметр для {model}:')
  print(GSCV.best_params_)
  return GSCV

def ErrCouner(model, GSCV, X_test, y_test, column):
  best_estimator = GSCV.best_estimator_
  result = pd.DataFrame({
     'Model': f'{model}', 
     'MAE': mean_absolute_error(y_test, best_estimator.predict(X_test)), 
     'MSE': mean_squared_error(y_test, best_estimator.predict(X_test)), 
     'R2 score': best_estimator.score(X_test, y_test)
  }, index=[column])
  print(result)
  return result

In [ ]:
# модель линейной регрессии

LRmodel = LinearRegression()
params = { 'fit_intercept' : ['True', 'False']}

In [ ]:
%%time
GSCV = GridSCV(LRmodel, params, X_train, y_train.values.ravel())
LRbest = GSCV.fit(X_train, y_train)
ErrCouner(LRmodel, GSCV, X_test, y_test, 'Модуль упругости при растяжении')

In [ ]:
# такие показатели говорят о том, что модель плохо работает 

In [ ]:
# модель "случайный лес"

RFmodel = RandomForestRegressor(random_state = 42)
params = {
   'n_estimators': [20, 40, 60],
   'max_features': ['auto', 'sqrt', 'log2'],
   'max_depth' : [3,4,5,6]
}

In [ ]:
%%time
GSCV = GridSCV(RFmodel, params, X_train, y_train.values.ravel())
RF_best = GSCV.fit(X_train, y_train)
ErrCouner(RFmodel, GSCV, X_test, y_test, 'Модуль упругости при растяжении')

In [ ]:
# такие показатели говорят о том, что модель плохо работает 

In [ ]:
# модель "Градиентный бустинг" 

GBmodel = GradientBoostingRegressor()
params = {
    'learning_rate': np.arange(0.001, 0.1, 0.003),
    'max_depth': np.arange(1, 12, 2),
    'max_features':['log2','sqrt'],    
          }

In [ ]:
%%time
GSCV = GridSCV(GBmodel, params, X_train, y_train.values.ravel())
GB_best = GSCV.fit(X_train, y_train)
ErrCouner(GBmodel, GSCV, X_test, y_test, 'Модуль упругости при растяжении')

In [ ]:
# ни одна из представленных моделей не дает эффективного результата предсказания

2. Модель для прогноза прочности при растяжении

In [ ]:
X = df_norm.drop('Прочность при растяжении, МПа', axis = 1)
y = df_norm['Прочность при растяжении, МПа']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)
print(f'Размер обучающей выборки: {X_train.shape[0]}')
print(f'Размер тестовой выборки: {X_test.shape[0]}')

In [ ]:
# модель линейной регрессии

LRmodel = LinearRegression()
params = { 'fit_intercept' : ['True', 'False']}

In [ ]:
%%time
GSCV = GridSCV(LRmodel, params, X_train, y_train.values.ravel())
LRbest = GSCV.fit(X_train, y_train)
ErrCouner(LRmodel, GSCV, X_test, y_test, 'Прочность при растяжении')

In [ ]:
# модель "случайный лес"

RFmodel = RandomForestRegressor(random_state = 42)
params = {
   'n_estimators': [20, 40, 60],
   'max_features': ['auto', 'sqrt', 'log2'],
   'max_depth' : [3,4,5,6]
}

In [ ]:
%%time
GSCV = GridSCV(RFmodel, params, X_train, y_train.values.ravel())
RF_best = GSCV.fit(X_train, y_train)
ErrCouner(RFmodel, GSCV, X_test, y_test, 'Прочность при растяжении')

In [ ]:
# модель "Градиентный бустинг" 
GBmodel = GradientBoostingRegressor()
params = {
    'learning_rate': np.arange(0.001, 0.1, 0.003),
    'max_depth': np.arange(1, 12, 2),
    'max_features':['log2','sqrt'],    
          }

In [ ]:
%%time
GSCV = GridSCV(GBmodel, params, X_train, y_train.values.ravel())
GB_best = GSCV.fit(X_train, y_train)
ErrCouner(GBmodel, GSCV, X_test, y_test, 'Прочность при растяжении')

In [ ]:
# отработавшие модели так же не показали положительный результат предсказаний

Нейронная сеть для рекомендации соотношение матрица-наполнитель

In [ ]:
# датасет с входными и выходными данными

df_clean = df_clean.reset_index(drop = True)
in_data = df_clean.drop('Соотношение матрица-наполнитель', axis = 1)
out_data = df_clean['Соотношение матрица-наполнитель']

In [ ]:
in_data

In [ ]:
out_data

In [ ]:
# деление датасета на обучающую и тестовую выборки

X_train, X_test, y_train, y_test = train_test_split(in_data, out_data, test_size = 0.2, random_state = 1)
print(f'Размер обучающей выборки: {X_train.shape[0]}')
print(f'Размер тестовой выборки: {X_test.shape[0]}')

In [ ]:
normalizer = tf.keras.layers.Normalization(input_shape = [12,], axis = None)
normalizer.adapt(np.array(X_train))

In [ ]:
X_train

In [ ]:
# создание модели

sqntl_model = Sequential([
                    normalizer,
                    Dense(1024, activation = 'relu'),
                    Dense(1024, activation = 'relu'),
                    Dense(1)
                    ])

sqntl_model.compile(loss = 'mean_squared_error', optimizer = tf.keras.optimizers.Adam(0.0005))

In [ ]:
sqntl_model.summary()

In [ ]:
# обучение модели

In [ ]:
%%time
history = sqntl_model.fit(X_train, y_train, validation_split = 0.2, verbose = 1, epochs = 70)

In [ ]:
# график для визуализации ошибки нейросети

def plot_loss(history):
    plt.plot(history.history['loss'], label = 'loss')
    plt.plot(history.history['val_loss'], label = 'val_loss')
    plt.ylim([0, 2])
    plt.xlabel('Эпоха')
    plt.ylabel('MSE [MPG]')
    plt.legend()
    plt.grid(True)

In [ ]:
plot_loss(history)

In [ ]:
# график рассеяния предстказанных данных нейросетью

import plotly.express as px

result_data = pd.DataFrame()
result_data["Предсказанное значение"] = sqntl_model.predict(np.array((X_test))).reshape(-1, )
result_data["Оригинальное значение"] = y_test.values

fig = px.scatter(result_data, x="Предсказанное значение", y="Оригинальное значение", trendline="ols")